In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import keras

In [2]:
model = keras.models.load_model('./Conv1D_and_LSTM_model')

In [3]:
scaler = MinMaxScaler(feature_range = (0,10))
def scaling_input_data(input_data):
    
    for i in range(0,len(input_data)):
        # To scale prices properly it's necessary to stack all price related features in one array before scaling
        concatenated_open_high_low_close = np.vstack((input_data[i][:,0],
                                                      input_data[i][:,1],
                                                      input_data[i][:,2],
                                                      input_data[i][:,3])).reshape(800,1)
        # Scaling the concatenated price data    
        scaler_Mod = scaler.fit(concatenated_open_high_low_close)
        scaled_concatenated_open_high_low_close = scaler_Mod.transform(concatenated_open_high_low_close)
        # Unstacking scaled values to achieve demanded shape of an array
        input_data[i][:,0] = scaled_concatenated_open_high_low_close[:200].reshape(200,)
        input_data[i][:,1] = scaled_concatenated_open_high_low_close[200:400].reshape(200,)
        input_data[i][:,2] = scaled_concatenated_open_high_low_close[400:600].reshape(200,)
        input_data[i][:,3] = scaled_concatenated_open_high_low_close[600:800].reshape(200,)
        # Scaling volume values
        scaler_Mod_2 = scaler.fit(input_data[i][:,4].reshape(200,1))
        scaled_volume = scaler_Mod_2.transform(input_data[i][:,4].reshape(200,1))
        # Updating main array with scaled volume
        input_data[i][:,4] = scaled_volume.reshape(200,)
    return 

In [4]:
def extraction_of_200_days_period_stocks_data(folder_name):
    # Getting a list of stock symbols from the folder
    ticker_list = list(os.listdir(folder_name))
    tickers_data = []
    # Extracting stocks data to a Data Frame
    for ticker in ticker_list:
        try:
            tickers_data.append(pd.read_csv(f'./{folder_name}/{ticker}', usecols=['<TICKER>','<OPEN>', '<HIGH>',
                                                                                  '<LOW>', '<CLOSE>','<VOL>']))
        except:
            pass
    # Creating a list of stock data for stocks tha have at least 200 days history
    ticker_with_least_200_days_history = []
    for ticker in tickers_data:
        if len(ticker) > 200:
            ticker_with_least_200_days_history.append(ticker)
    # Creating a list of stock data with (200, 5) shape        
    ticker_data_200_days = []
    for ticker in ticker_with_least_200_days_history:
        ticker_data_200_days.append(ticker.iloc[-200:,1:])
    ticker_data_200_days_array = np.asarray(ticker_data_200_days)
    return ticker_data_200_days_array, ticker_with_least_200_days_history, ticker_data_200_days

In [5]:
def show_selected_stocks(model_name, stock_data_array, stock_data_before_preprocessing, ticker_data_200_days, min_probability):
    selected_stocks = pd.DataFrame(model_name.predict(stock_data_array, verbose = 0))
    selected_stocks.columns = ['probability_of_positive_class']
    index_of_selected_stocks = selected_stocks[selected_stocks['probability_of_positive_class']>min_probability].index
    list_of_selected_stocks = []
    for index in index_of_selected_stocks:
        if ticker_data_200_days[index]['<VOL>'].max() > 40000:
            list_of_selected_stocks.append(stock_data_before_preprocessing[index].iloc[0,0])
    return list_of_selected_stocks

In [6]:
Folder_name = 'NYSE_data'
ticker_data_200_days_array, ticker_with_least_200_days_history, ticker_data_200_days \
= extraction_of_200_days_period_stocks_data(Folder_name)
scaling_input_data(ticker_data_200_days_array)
show_selected_stocks(model, ticker_data_200_days_array, ticker_with_least_200_days_history, ticker_data_200_days, 0.5)

['AEVA-WS.US',
 'ALLY.US',
 'AMRC.US',
 'APTV.US',
 'AWF.US',
 'AXL.US',
 'BBN.US',
 'BC.US',
 'BDJ.US',
 'BEP.US',
 'BSTZ.US',
 'BTZ.US',
 'CII.US',
 'COF_H.US',
 'COF_J.US',
 'CRHC-WS.US',
 'CWH.US',
 'CX.US',
 'C_J.US',
 'DFP.US',
 'DTLA_.US',
 'ECL.US',
 'EFR.US',
 'EFT.US',
 'EMD.US',
 'EOS.US',
 'ETV.US',
 'ETX.US',
 'FNGO.US',
 'FNGS.US',
 'FNGU.US',
 'FTV.US',
 'GRMN.US',
 'HASI.US',
 'HCI.US',
 'HFRO_A.US',
 'HIO.US',
 'HYT.US',
 'HZO.US',
 'IGI.US',
 'JPS.US',
 'MFL.US',
 'MG.US',
 'MHF.US',
 'MKFG.US',
 'MPA.US',
 'MUE.US',
 'MYD.US',
 'NEV.US',
 'NKG.US',
 'NMCO.US',
 'NQP.US',
 'NRGD.US',
 'NUO.US',
 'NXJ.US',
 'OIA.US',
 'PCN.US',
 'PDI.US',
 'PEI_B.US',
 'PHG.US',
 'PIAI-WS.US',
 'PMM.US',
 'PNF.US',
 'PZC.US',
 'RFI.US',
 'RVI.US',
 'SAR.US',
 'SHOP.US',
 'SMHB.US',
 'STOR.US',
 'THO.US',
 'THQ.US',
 'VPG.US',
 'VTN.US',
 'W.US',
 'WGO.US',
 'XYL.US',
 'YSAC.US']